# String simulator

In [9]:
from matplotlib import pyplot as plt
from matplotlib import animation as ani
import numpy as np
import scipy as sp

import time
import pickle

In [10]:
print("Last run: " + time.ctime())

Last run: Tue Jan 30 13:41:25 2024


## Setting up the solver

***
We are going to need to make some modifications to our system of equations to work with `solve_ivp`.

First, `solve_ivp` only operates on first-order ODEs. We can convert our system of $n$ 2nd-order ODEs into a systen of $2n$ 1st-order ODEs with the following substitutions:

$$v_i=\frac{du_i}{dt}$$

$$\frac{dv_i}{dt}=\frac{T}{\rho}\frac{u_{i+1}+u_{i-1}-2u_i}{\Delta x^2}-\mu_i v_i$$

We'll vectorize the system of $u$ and $v$, into a vector $p$, such that $p_i=u_i$ for $1\leq i\leq n$, and $p_i=v_{i-500}$ for $n+1\leq i\leq 2n$:

$$p=\left[u_1, u_2, u_3, \dots u_n, v_1, v_2, v_3, \dots v_n\right]$$

Its derivative (using dot notation for $\frac{d}{dt}$) is:

$$\dot{p}=\left[\dot{u}_1, \dot{u}_2, \dot{u}_3, \dots \dot{u}_n, \dot{v}_1, \dot{v}_2, \dot{v}_3, \dots \dot{v}_n\right]$$

We will relate $p$ and $\dot{p}$ with a matrix of coefficients, $C$:

$$\dot{p}=pC$$

The coefficient $c_{ij}$ in $C$ should relate $p_i$ with $\dot{p}_j$:

$$\dot{p}_j=\sum_{i}p_ic_{ij}$$

Substituting $u_i$ with $p_i$ and $v_i$ with $p_{i+n}$:

$$p_{i+n}=\frac{dp_i}{dt}$$

$$\frac{dp_{i+n}}{dt}=\frac{T}{\rho}\frac{p_{i+1}+p_{i-1}-2p_i}{\Delta x^2}-\mu_i p_{i+n}$$

$$c_{ij}=\begin{cases}
    0&\text{for }j=n+1\text{ or }j=2n\\
    1&\text{for }i=j+n\\
    \frac{T}{\Delta x^2\rho}&\text{for }j=i+n-1\text{ or }j=i+n+1\\
    \frac{-2T}{\Delta x^2\rho}&\text{for }j=i+n\\
    -\mu_{i-n}&\text{for }i=j\text{ and }i>n
\end{cases}$$
This gives a system of $2n$ first-order ODEs which we can solve with `solve_ivp`.

In [11]:
T = 103
r = 0.0270
L = 0.75
#fundamental frequency according to mersenne's laws:
print("f0 = {0:.2f}".format(0.5*np.sqrt(T/r)/L))

f0 = 41.18


In [12]:
def simulate(tension, density, length, damping, initial, n_points=500):
    """simulate a string given length, tension, linear density
    All quantities should be given in SI units -
    tension should have units of N,
    density should have units of kg/m,
    length should have units of m,
    time should have units of seconds
    """

    if damping.shape[0] != n_points:
        raise ValueError("Damping vector must have length n_points")
    elif initial.shape[0] != 2*n_points:
        raise ValueError("Initial condition vector must have length 2*n_points")
    
    c = np.zeros((2*n_points, 2*n_points))

    dx = length/(n_points-1)

    st = time.time()
    print("Beginning...")

    #create c matrix according to rules above
    for i in range(0, 2*n_points):
        for j in range(0, 2*n_points):
            if j==n_points or j==2*n_points-1:
                c[i,j] = 0
            elif i == j+n_points:
                c[i,j]=1
            elif j==i+n_points-1 or j==i+n_points+1:
                c[i,j]=tension/(dx**2*density)
            elif j == i+n_points:
                c[i,j]=-2*tension/(dx**2*density)
            elif i==j and i > n_points:
                c[i,j] = -damping[i-n_points]

    print("Created c matrix in {0:.2f} s".format(time.time()-st))
          
    def func(t, y):
        return np.matmul(y, c)

    t_eval = np.linspace(0, 1, 10000)

    #way faster with 2(3)
    s = sp.integrate.solve_ivp(func, (0,1), initial, t_eval=t_eval, method="RK23")
    
    print("Solver finished in {0:.2f} s".format(time.time()-st))
          
    return s

In [13]:
p0_p = np.zeros(1000)
p0_n = np.zeros(1000)

p0_p[0:450]=[0.03*i/450 for i in range(0,450)]
p0_p[450:499]=[0.03*(49-i)/49 for i in range(0,49)]

p0_n[0:300]=[0.03*i/300 for i in range(0,300)]
p0_n[300:499]=[0.03*(199-i)/199 for i in range(0,199)]

In [14]:
mu_open = np.full(500, 0.5)

mu_palm = np.full(500, 0.5)
mu_palm[-50:] = 200

mu_har2 = np.full(500, 0.5)
mu_har2[249:251] = 5000

mu_har3 = np.full(500, 0.5)
mu_har3[166:168] = 5000

In [15]:
s_op_p = simulate(T, r, L, mu_open, p0_p) #open string, plucked near the pickup
s_op_n = simulate(T, r, L, mu_open, p0_n) #open string, plucked near the neck
s_palm = simulate(T, r, L, mu_palm, p0_p) #palm muted string, plucked near the pickup
s_har2 = simulate(T, r, L, mu_har2, p0_p) #string muted for 2nd harmonic
s_har3 = simulate(T, r, L, mu_har3, p0_p) #string muted for 3rd harmonic

Beginning...
Created c matrix in 0.60 s


KeyboardInterrupt: 

In [ ]:
with open("op_p.dat", "wb") as f:
    pickle.dump(s_op_p, f)

with open("op_n.dat", "wb") as f:
    pickle.dump(s_op_n, f)

with open("palm.dat", "wb") as f:
    pickle.dump(s_palm, f)

with open("har2.dat", "wb") as f:
    pickle.dump(s_har2, f)

with open("har3.dat", "wb") as f:
    pickle.dump(s_har3, f)